In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Model Building

### Get the cleaned and pivotted files for each measure category
### Apply weights (multiply by 22 or 4 as per deloite case study
### Merge the Data frames on provider id 
### Get the target variable from Hospital General Information file - 'Hospital Overall Rating'
### Apply Quantile transformer to normalize the data 
### Apply RFE and Build Liniear model

In [2]:
df1=pd.read_csv("Efficiency_score.csv")
df2=pd.read_csv("Mortality_score.csv")
df3=pd.read_csv("Readmission_score.csv")

In [8]:
df1.head(1)
df2.head(1)
df3.head(1)

,Provider ID,OP_10,OP_11,OP_13,OP_14,OP_8,Efficiency_score
0,10001,4.416,0.144,-0.071,-0.048,3.048,1.4978


In [9]:
df1=df1.drop('Unnamed: 0',axis=1)
df2=df2.drop('Unnamed: 0',axis=1)
df3=df3.drop('Unnamed: 0',axis=1)

In [21]:
df1.iloc[:,1:] = df1.iloc[:,1:] * 4
df1.head()

,Provider ID,OP_10,OP_11,OP_13,OP_14,OP_8,Efficiency_score
0,10001,17.664,0.576,-0.284,-0.192,12.192,5.9912
1,10005,36.708,6.528,-0.212,-0.320,13.696,11.2800
2,10006,32.568,2.944,-0.000,-0.368,14.592,9.9472
3,10007,67.068,3.904,-0.000,-0.000,0.000,14.1944
4,10008,9.936,0.000,-0.000,-0.048,0.000,1.9776


In [22]:
df2.iloc[:,1:] = df2.iloc[:,1:] * 22
df2.head()

,Provider ID,MORT_30_AMI,MORT_30_CABG,MORT_30_COPD,MORT_30_HF,MORT_30_PN,MORT_30_STK,Mort_score
0,10001,132.000,32.34,126.852,201.872,225.060,176.176,149.050000
1,10005,168.960,0.00,103.664,252.340,302.016,177.320,167.383333
2,10006,176.352,31.57,96.844,253.968,264.264,204.776,171.295667
3,10007,0.000,0.00,126.852,234.432,272.976,189.904,137.360667
4,10008,0.000,0.00,111.848,206.756,227.964,0.000,91.094667


In [23]:
df3.iloc[:,1:] = df3.iloc[:,1:] * 22
df3.head()

,Provider ID,READM_30_AMI,READM_30_CABG,READM_30_COPD,READM_30_HF,READM_30_HIP_KNEE,READM_30_HOSP_WIDE,READM_30_PN,READM_30_STK,Readmission_score
0,10001,196.020,89.694,264.594,296.604,47.124,318.472,250.954,148.082,201.44300
1,10005,198.396,0.000,225.720,303.534,52.668,308.132,220.088,156.244,183.09775
2,10006,191.268,90.288,248.292,285.516,46.200,318.472,240.218,139.920,195.02175
3,10007,0.000,0.000,249.546,292.446,0.000,343.288,232.166,148.082,158.19100
4,10008,0.000,0.000,240.768,320.166,0.000,324.676,214.720,0.000,137.54125


In [193]:
df= pd.merge(df1, df2, on='Provider ID', how='outer')

In [194]:
df= pd.merge(df, df3, on='Provider ID', how='outer')

In [129]:
df.shape

(4447, 23)

In [195]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4447 entries, 0 to 4446
Data columns (total 23 columns):
Provider ID           4447 non-null int64
OP_10                 3779 non-null float64
OP_11                 3779 non-null float64
OP_13                 3779 non-null float64
OP_14                 3779 non-null float64
OP_8                  3779 non-null float64
Efficiency_score      3779 non-null float64
MORT_30_AMI           4112 non-null float64
MORT_30_CABG          4112 non-null float64
MORT_30_COPD          4112 non-null float64
MORT_30_HF            4112 non-null float64
MORT_30_PN            4112 non-null float64
MORT_30_STK           4112 non-null float64
Mort_score            4112 non-null float64
READM_30_AMI          4405 non-null float64
READM_30_CABG         4405 non-null float64
READM_30_COPD         4405 non-null float64
READM_30_HF           4405 non-null float64
READM_30_HIP_KNEE     4405 non-null float64
READM_30_HOSP_WIDE    4405 non-null float64
READM_30_PN    

In [29]:
# Check the data for percentage of null values

round((df.isnull().sum()/df.isnull().count())*100,2)

Provider ID            0.00
OP_10                 15.02
OP_11                 15.02
OP_13                 15.02
OP_14                 15.02
OP_8                  15.02
Efficiency_score      15.02
MORT_30_AMI            7.53
MORT_30_CABG           7.53
MORT_30_COPD           7.53
MORT_30_HF             7.53
MORT_30_PN             7.53
MORT_30_STK            7.53
Mort_score             7.53
READM_30_AMI           0.94
READM_30_CABG          0.94
READM_30_COPD          0.94
READM_30_HF            0.94
READM_30_HIP_KNEE      0.94
READM_30_HOSP_WIDE     0.94
READM_30_PN            0.94
READM_30_STK           0.94
Readmission_score      0.94
dtype: float64

In [196]:
df.fillna(0, inplace=True)

In [119]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4447 entries, 0 to 4446
Data columns (total 23 columns):
Provider ID           4447 non-null int64
OP_10                 4447 non-null float64
OP_11                 4447 non-null float64
OP_13                 4447 non-null float64
OP_14                 4447 non-null float64
OP_8                  4447 non-null float64
Efficiency_score      4447 non-null float64
MORT_30_AMI           4447 non-null float64
MORT_30_CABG          4447 non-null float64
MORT_30_COPD          4447 non-null float64
MORT_30_HF            4447 non-null float64
MORT_30_PN            4447 non-null float64
MORT_30_STK           4447 non-null float64
Mort_score            4447 non-null float64
READM_30_AMI          4447 non-null float64
READM_30_CABG         4447 non-null float64
READM_30_COPD         4447 non-null float64
READM_30_HF           4447 non-null float64
READM_30_HIP_KNEE     4447 non-null float64
READM_30_HOSP_WIDE    4447 non-null float64
READM_30_PN    

In [32]:
df_target=pd.read_csv("Hospital General Information_C.csv")
df_target.head()

,Provider ID,Hospital Name,ZIP Code,County Name,Hospital Type,Hospital Ownership,Emergency Services,Hospital overall rating,Mortality national comparison,Safety of care national comparison,Readmission national comparison,Patient experience national comparison,Effectiveness of care national comparison,Timeliness of care national comparison,Efficient use of medical imaging national comparison
0,10001,SOUTHEAST ALABAMA MEDICAL CENTER,36301,HOUSTON,Acute Care Hospitals,Government - Hospital District or Authority,1,3,2,3,2,1,2,2,2
1,10005,MARSHALL MEDICAL CENTER SOUTH,35957,MARSHALL,Acute Care Hospitals,Government - Hospital District or Authority,1,3,1,2,3,2,2,3,1
2,10006,ELIZA COFFEE MEMORIAL HOSPITAL,35631,LAUDERDALE,Acute Care Hospitals,Government - Hospital District or Authority,1,2,1,2,2,1,2,3,2
3,10011,ST VINCENT'S EAST,35235,JEFFERSON,Acute Care Hospitals,Voluntary non-profit - Private,1,2,2,1,2,1,1,2,2
4,10012,DEKALB REGIONAL MEDICAL CENTER,35968,DE KALB,Acute Care Hospitals,Proprietary,1,3,1,2,2,2,2,3,2


In [33]:
dft=df_target[['Provider ID','Hospital overall rating']]
dft.head()

,Provider ID,Hospital overall rating
0,10001,3
1,10005,3
2,10006,2
3,10011,2
4,10012,3


In [197]:
df= pd.merge(df, dft, on='Provider ID', how='outer')

In [136]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2333 entries, 0 to 3772
Data columns (total 24 columns):
Provider ID                2333 non-null int64
OP_10                      2333 non-null float64
OP_11                      2333 non-null float64
OP_13                      2333 non-null float64
OP_14                      2333 non-null float64
OP_8                       2333 non-null float64
Efficiency_score           2333 non-null float64
MORT_30_AMI                2333 non-null float64
MORT_30_CABG               2333 non-null float64
MORT_30_COPD               2333 non-null float64
MORT_30_HF                 2333 non-null float64
MORT_30_PN                 2333 non-null float64
MORT_30_STK                2333 non-null float64
Mort_score                 2333 non-null float64
READM_30_AMI               2333 non-null float64
READM_30_CABG              2333 non-null float64
READM_30_COPD              2333 non-null float64
READM_30_HF                2333 non-null float64
READM_30_HIP_

In [36]:
df['Hospital overall rating'].unique()

array([ 3.,  2., nan,  4.,  5.,  1.])

In [198]:
# Dropping the records where we dont have the target ratings available
df= df.dropna()

In [199]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2333 entries, 0 to 3772
Data columns (total 24 columns):
Provider ID                2333 non-null int64
OP_10                      2333 non-null float64
OP_11                      2333 non-null float64
OP_13                      2333 non-null float64
OP_14                      2333 non-null float64
OP_8                       2333 non-null float64
Efficiency_score           2333 non-null float64
MORT_30_AMI                2333 non-null float64
MORT_30_CABG               2333 non-null float64
MORT_30_COPD               2333 non-null float64
MORT_30_HF                 2333 non-null float64
MORT_30_PN                 2333 non-null float64
MORT_30_STK                2333 non-null float64
Mort_score                 2333 non-null float64
READM_30_AMI               2333 non-null float64
READM_30_CABG              2333 non-null float64
READM_30_COPD              2333 non-null float64
READM_30_HF                2333 non-null float64
READM_30_HIP_

In [200]:
from sklearn.preprocessing import QuantileTransformer

In [201]:
df_bkp=df
df_bkp.head()


,Provider ID,OP_10,OP_11,OP_13,OP_14,OP_8,Efficiency_score,MORT_30_AMI,MORT_30_CABG,MORT_30_COPD,...,READM_30_AMI,READM_30_CABG,READM_30_COPD,READM_30_HF,READM_30_HIP_KNEE,READM_30_HOSP_WIDE,READM_30_PN,READM_30_STK,Readmission_score,Hospital overall rating
0,10001,17.664,0.576,-0.284,-0.192,12.192,5.9912,132.000,32.34,126.852,...,196.020,89.694,264.594,296.604,47.124,318.472,250.954,148.082,201.44300,3.0
1,10005,36.708,6.528,-0.212,-0.320,13.696,11.2800,168.960,0.00,103.664,...,198.396,0.000,225.720,303.534,52.668,308.132,220.088,156.244,183.09775,3.0
2,10006,32.568,2.944,-0.000,-0.368,14.592,9.9472,176.352,31.57,96.844,...,191.268,90.288,248.292,285.516,46.200,318.472,240.218,139.920,195.02175,2.0
5,10011,6.624,6.656,-0.000,-0.272,11.584,4.9184,146.784,28.49,100.936,...,210.276,89.100,245.784,274.428,47.124,316.404,224.114,131.758,192.37350,2.0
6,10012,16.836,6.912,-0.124,-0.000,0.000,4.7248,177.408,0.00,125.488,...,191.268,0.000,240.768,324.324,0.000,316.404,260.348,149.248,185.29500,3.0


In [192]:
df=df_bkp
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2333 entries, 0 to 3772
Data columns (total 24 columns):
Provider ID                2333 non-null int64
OP_10                      1419 non-null float64
OP_11                      1419 non-null float64
OP_13                      1419 non-null float64
OP_14                      1419 non-null float64
OP_8                       1419 non-null float64
Efficiency_score           1419 non-null float64
MORT_30_AMI                1419 non-null float64
MORT_30_CABG               1419 non-null float64
MORT_30_COPD               1419 non-null float64
MORT_30_HF                 1419 non-null float64
MORT_30_PN                 1419 non-null float64
MORT_30_STK                1419 non-null float64
Mort_score                 1419 non-null float64
READM_30_AMI               1419 non-null float64
READM_30_CABG              1419 non-null float64
READM_30_COPD              1419 non-null float64
READM_30_HF                1419 non-null float64
READM_30_HIP_

In [185]:
dfc=df.drop(['Provider ID'],axis=1)
list=dfc.columns.tolist()

In [202]:
dfn=df_bkp.drop(['Provider ID'],axis=1)

In [203]:
list

['OP_10',
 'OP_11',
 'OP_13',
 'OP_14',
 'OP_8',
 'Efficiency_score',
 'MORT_30_AMI',
 'MORT_30_CABG',
 'MORT_30_COPD',
 'MORT_30_HF',
 'MORT_30_PN',
 'MORT_30_STK',
 'Mort_score',
 'READM_30_AMI',
 'READM_30_CABG',
 'READM_30_COPD',
 'READM_30_HF',
 'READM_30_HIP_KNEE',
 'READM_30_HOSP_WIDE',
 'READM_30_PN',
 'READM_30_STK',
 'Readmission_score',
 'Hospital overall rating']

In [204]:
qnt= QuantileTransformer(output_distribution='normal')
dfn[list]=pd.DataFrame(qnt.fit_transform(df[list]))

In [66]:
dfa=pd.DataFrame(qnt.fit_transform(df.iloc[:,1:]))

In [209]:
dfn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2333 entries, 0 to 3772
Data columns (total 23 columns):
OP_10                      1419 non-null float64
OP_11                      1419 non-null float64
OP_13                      1419 non-null float64
OP_14                      1419 non-null float64
OP_8                       1419 non-null float64
Efficiency_score           1419 non-null float64
MORT_30_AMI                1419 non-null float64
MORT_30_CABG               1419 non-null float64
MORT_30_COPD               1419 non-null float64
MORT_30_HF                 1419 non-null float64
MORT_30_PN                 1419 non-null float64
MORT_30_STK                1419 non-null float64
Mort_score                 1419 non-null float64
READM_30_AMI               1419 non-null float64
READM_30_CABG              1419 non-null float64
READM_30_COPD              1419 non-null float64
READM_30_HF                1419 non-null float64
READM_30_HIP_KNEE          1419 non-null float64
READM_30_HO

In [210]:
dfn=dfn.dropna()

In [103]:
dff = pd.concat([dfn,df['Hospital overall rating']],axis=1)

In [211]:
from sklearn.model_selection import train_test_split


df_train, df_test = train_test_split(dfn, train_size = 0.7, test_size = 0.3, random_state = 100)

In [212]:
y_train = df_train.pop('Hospital overall rating')
X_train = df_train

In [213]:
# Importing RFE and LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

# Running RFE with the output number of the variable equal to 10
lm = LinearRegression()
lm.fit(X_train, y_train)

rfe = RFE(lm, 10)             # running RFE
rfe = rfe.fit(X_train, y_train)


In [214]:
#Get the columns selected by rfe
col = X_train.columns[rfe.support_]
col

Index(['OP_10', 'OP_8', 'Efficiency_score', 'MORT_30_COPD', 'MORT_30_HF',
       'MORT_30_PN', 'Mort_score', 'READM_30_HF', 'READM_30_HOSP_WIDE',
       'Readmission_score'],
      dtype='object')

In [215]:
# Creating X_test dataframe with RFE selected variables
X_train_rfe = X_train[col]

# Adding a constant variable
import statsmodels.api as sm  
X_train_rfe = sm.add_constant(X_train_rfe)

In [216]:
lm = sm.OLS(y_train,X_train_rfe).fit()   # Running the linear model

In [217]:
#Let's see the summary of our linear model
print(lm.summary())

                               OLS Regression Results                              
Dep. Variable:     Hospital overall rating   R-squared:                       0.430
Model:                                 OLS   Adj. R-squared:                  0.424
Method:                      Least Squares   F-statistic:                     74.14
Date:                     Fri, 06 Sep 2019   Prob (F-statistic):          9.89e-113
Time:                             04:09:36   Log-Likelihood:                -1587.9
No. Observations:                      993   AIC:                             3198.
Df Residuals:                          982   BIC:                             3252.
Df Model:                               10                                         
Covariance Type:                 nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------

In [220]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

In [221]:
lr.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [223]:
!pip install eli5

  Running setup.py bdist_wheel for tabulate: started
  Running setup.py bdist_wheel for tabulate: finished with status 'done'
  Stored in directory: C:\Users\199251\AppData\Local\pip\Cache\wheels\2b\67\89\414471314a2d15de625d184d8be6d38a03ae1e983dbda91e84
Successfully built tabulate


You are using pip version 9.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [226]:
import eli5
eli5.show_weights(lr,feature_names=['OP_10',
 'OP_11',
 'OP_13',
 'OP_14',
 'OP_8',
 'Efficiency_score',
 'MORT_30_AMI',
 'MORT_30_CABG',
 'MORT_30_COPD',
 'MORT_30_HF',
 'MORT_30_PN',
 'MORT_30_STK',
 'Mort_score',
 'READM_30_AMI',
 'READM_30_CABG',
 'READM_30_COPD',
 'READM_30_HF',
 'READM_30_HIP_KNEE',
 'READM_30_HOSP_WIDE',
 'READM_30_PN',
 'READM_30_STK',
 'Readmission_score'])

# Random Forrest Model

In [230]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2333 entries, 0 to 3772
Data columns (total 24 columns):
Provider ID                2333 non-null int64
OP_10                      2333 non-null float64
OP_11                      2333 non-null float64
OP_13                      2333 non-null float64
OP_14                      2333 non-null float64
OP_8                       2333 non-null float64
Efficiency_score           2333 non-null float64
MORT_30_AMI                2333 non-null float64
MORT_30_CABG               2333 non-null float64
MORT_30_COPD               2333 non-null float64
MORT_30_HF                 2333 non-null float64
MORT_30_PN                 2333 non-null float64
MORT_30_STK                2333 non-null float64
Mort_score                 2333 non-null float64
READM_30_AMI               2333 non-null float64
READM_30_CABG              2333 non-null float64
READM_30_COPD              2333 non-null float64
READM_30_HF                2333 non-null float64
READM_30_HIP_

In [231]:
# split into X and y
X = df.loc[:, df.columns != 'Hospital overall rating']

y = df['Hospital overall rating']

In [232]:
# scaling the features
from sklearn.preprocessing import scale

cols = X.columns
X = pd.DataFrame(scale(X))
X.columns = cols
X.columns

Index(['Provider ID', 'OP_10', 'OP_11', 'OP_13', 'OP_14', 'OP_8',
       'Efficiency_score', 'MORT_30_AMI', 'MORT_30_CABG', 'MORT_30_COPD',
       'MORT_30_HF', 'MORT_30_PN', 'MORT_30_STK', 'Mort_score', 'READM_30_AMI',
       'READM_30_CABG', 'READM_30_COPD', 'READM_30_HF', 'READM_30_HIP_KNEE',
       'READM_30_HOSP_WIDE', 'READM_30_PN', 'READM_30_STK',
       'Readmission_score'],
      dtype='object')

In [233]:
# split into train and test
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size=0.7,
                                                    test_size = 0.3, random_state=100)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [234]:
# Importing random forest classifier from sklearn library
from sklearn.ensemble import RandomForestClassifier

# Running the random forest with default parameters.
rfc = RandomForestClassifier()

In [235]:
# fit
rfc.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [236]:
# Making predictions
predictions = rfc.predict(X_test)

In [237]:
# Importing classification report and confusion matrix from sklearn metrics
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score

In [238]:
# Let's check the report of our default model
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

        1.0       0.27      0.09      0.14        33
        2.0       0.48      0.43      0.45       163
        3.0       0.52      0.65      0.58       318
        4.0       0.48      0.40      0.44       171
        5.0       0.00      0.00      0.00        15

avg / total       0.48      0.50      0.48       700



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [239]:
print(accuracy_score(y_test,predictions))

0.49714285714285716
